In [ ]:
#랜덤포레스트(배깅 앙상블 알고리즘)
#트리모델들을 임의로 많이 만들어서 다수결투표로 결과를 종합하는 알고리즘
#2가지 임의요소 : 부트스트래핑
#모델을 똑같은 데이터셋으로 학습시키면 결과 다양성 떨어질수있음.
#모델을 임의로 만들어준 boostrap 데이터셋을 만들어내고 모델들의 결정을 합친다 (aggregating )==>bagging

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv("/content/drive/MyDrive/데청캠2022/2015_2019_기상_소방_화재데이터.csv" , encoding = "cp949")
df.head()

,Column1,사망,부상,인명피해(명)소계,재산피해소계,화재발생년월일,시도,시군구,화재유형,발화열원,일시,기온(°C),풍속(m/s),강수량(mm),소방서개수,안전센터개수,인구수,면적,인구밀도,소방인력
0,0,0,0,0,0,2015-01-01 0:06,서울특별시,영등포구,임야,"담뱃불, 라이터불",2015-01-01 0:00,-5.6,7.9,0.0,1,5,167863,24.55,6837.6,278
1,1,0,0,0,137,2015-01-01 0:07,서울특별시,강남구,"자동차,철도차량","담뱃불, 라이터불",2015-01-01 0:00,-5.7,2.9,0.0,1,6,237373,39.50,6009.4,322
2,2,0,0,0,326,2015-01-01 0:08,서울특별시,도봉구,"건축,구조물","담뱃불, 라이터불",2015-01-01 0:00,-6.9,2.5,0.0,1,4,136903,20.65,6629.7,215
3,3,0,0,0,30800,2015-01-01 0:34,서울특별시,용산구,"자동차,철도차량",미상(발화원인),2015-01-01 0:00,-6.0,3.3,0.0,1,5,107512,21.87,4916.0,250
4,4,0,0,0,11,2015-01-01 4:32,서울특별시,용산구,기타(쓰레기 화재등),"담뱃불, 라이터불",2015-01-01 4:00,-7.7,2.3,0.0,1,5,107512,21.87,4916.0,250


In [3]:
from sklearn.preprocessing import MinMaxScaler
df_score = df[["사망" , "부상" , "재산피해소계"]]
scaler = MinMaxScaler()
df_score_normalize = scaler.fit_transform(df_score)
df_score_normalize = pd.DataFrame(df_score_normalize , columns = df_score.columns)
df_score_normalize.describe()

,사망,부상,재산피해소계
count,30591.000000,30591.000000,3.059100e+04
mean,0.000906,0.001402,7.187792e-05
std,0.014674,0.011425,5.756723e-03
min,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,4.189164e-07
50%,0.000000,0.000000,1.536027e-06
75%,0.000000,0.000000,6.842301e-06
max,1.000000,1.000000,1.000000e+00


In [4]:
df["화재위험지수"] = 0.5* ( df_score_normalize ["사망"] * 0.8 + df_score_normalize["부상"]*0.2 ) +  0.5 * df_score_normalize["재산피해소계"] 

In [5]:
df.head()

,Column1,사망,부상,인명피해(명)소계,재산피해소계,화재발생년월일,시도,시군구,화재유형,발화열원,...,기온(°C),풍속(m/s),강수량(mm),소방서개수,안전센터개수,인구수,면적,인구밀도,소방인력,화재위험지수
0,0,0,0,0,0,2015-01-01 0:06,서울특별시,영등포구,임야,"담뱃불, 라이터불",...,-5.6,7.9,0.0,1,5,167863,24.55,6837.6,278,0.000000e+00
1,1,0,0,0,137,2015-01-01 0:07,서울특별시,강남구,"자동차,철도차량","담뱃불, 라이터불",...,-5.7,2.9,0.0,1,6,237373,39.50,6009.4,322,9.565258e-07
2,2,0,0,0,326,2015-01-01 0:08,서울특별시,도봉구,"건축,구조물","담뱃불, 라이터불",...,-6.9,2.5,0.0,1,4,136903,20.65,6629.7,215,2.276112e-06
3,3,0,0,0,30800,2015-01-01 0:34,서울특별시,용산구,"자동차,철도차량",미상(발화원인),...,-6.0,3.3,0.0,1,5,107512,21.87,4916.0,250,2.150437e-04
4,4,0,0,0,11,2015-01-01 4:32,서울특별시,용산구,기타(쓰레기 화재등),"담뱃불, 라이터불",...,-7.7,2.3,0.0,1,5,107512,21.87,4916.0,250,7.680134e-08


시간과 관련한 변수 추가

In [6]:
df['date'] = pd.to_datetime(df['화재발생년월일'], format='%Y-%m-%d %H:%M', errors='raise')  #datetime 형태로 변형하여 칼럼 추가
df['month']=df['date'].dt.month 
df['hour']=df['date'].dt.hour

In [7]:
df.head()

,Column1,사망,부상,인명피해(명)소계,재산피해소계,화재발생년월일,시도,시군구,화재유형,발화열원,...,소방서개수,안전센터개수,인구수,면적,인구밀도,소방인력,화재위험지수,date,month,hour
0,0,0,0,0,0,2015-01-01 0:06,서울특별시,영등포구,임야,"담뱃불, 라이터불",...,1,5,167863,24.55,6837.6,278,0.000000e+00,2015-01-01 00:06:00,1,0
1,1,0,0,0,137,2015-01-01 0:07,서울특별시,강남구,"자동차,철도차량","담뱃불, 라이터불",...,1,6,237373,39.50,6009.4,322,9.565258e-07,2015-01-01 00:07:00,1,0
2,2,0,0,0,326,2015-01-01 0:08,서울특별시,도봉구,"건축,구조물","담뱃불, 라이터불",...,1,4,136903,20.65,6629.7,215,2.276112e-06,2015-01-01 00:08:00,1,0
3,3,0,0,0,30800,2015-01-01 0:34,서울특별시,용산구,"자동차,철도차량",미상(발화원인),...,1,5,107512,21.87,4916.0,250,2.150437e-04,2015-01-01 00:34:00,1,0
4,4,0,0,0,11,2015-01-01 4:32,서울특별시,용산구,기타(쓰레기 화재등),"담뱃불, 라이터불",...,1,5,107512,21.87,4916.0,250,7.680134e-08,2015-01-01 04:32:00,1,4


소방과 관련한 변수 살펴보기

In [8]:
df["소방서개수"].value_counts()

1    30591
Name: 소방서개수, dtype: int64

In [9]:
df["안전센터개수"].value_counts()

6    13564
4     9638
5     5548
3     1054
2      787
Name: 안전센터개수, dtype: int64

결측값처리

In [10]:
df = df.dropna(axis =0)

변수선택

In [11]:
data = df[["시군구","화재유형"  ,	"발화열원" , "안전센터개수"	,"인구수",	"면적",	"인구밀도",	"기온(°C)",	"풍속(m/s)",	"강수량(mm)", "month" ,	"hour",  "화재위험지수"]]
data.head()

,시군구,화재유형,발화열원,안전센터개수,인구수,면적,인구밀도,기온(°C),풍속(m/s),강수량(mm),month,hour,화재위험지수
0,영등포구,임야,"담뱃불, 라이터불",5,167863,24.55,6837.6,-5.6,7.9,0.0,1,0,0.000000e+00
1,강남구,"자동차,철도차량","담뱃불, 라이터불",6,237373,39.50,6009.4,-5.7,2.9,0.0,1,0,9.565258e-07
2,도봉구,"건축,구조물","담뱃불, 라이터불",4,136903,20.65,6629.7,-6.9,2.5,0.0,1,0,2.276112e-06
3,용산구,"자동차,철도차량",미상(발화원인),5,107512,21.87,4916.0,-6.0,3.3,0.0,1,0,2.150437e-04
4,용산구,기타(쓰레기 화재등),"담뱃불, 라이터불",5,107512,21.87,4916.0,-7.7,2.3,0.0,1,4,7.680134e-08


In [12]:
data = pd.get_dummies(data)
data.head()

,안전센터개수,인구수,면적,인구밀도,기온(°C),풍속(m/s),강수량(mm),month,hour,화재위험지수,...,발화열원_기타(발화원인),"발화열원_담뱃불, 라이터불","발화열원_마찰, 전도, 복사",발화열원_미상,발화열원_미상(발화원인),"발화열원_불꽃, 불티",발화열원_자연적 발화열,발화열원_작동기기,"발화열원_폭발물, 폭죽",발화열원_화학적 발화열
0,5,167863,24.55,6837.6,-5.6,7.9,0.0,1,0,0.000000e+00,...,0,1,0,0,0,0,0,0,0,0
1,6,237373,39.50,6009.4,-5.7,2.9,0.0,1,0,9.565258e-07,...,0,1,0,0,0,0,0,0,0,0
2,4,136903,20.65,6629.7,-6.9,2.5,0.0,1,0,2.276112e-06,...,0,1,0,0,0,0,0,0,0,0
3,5,107512,21.87,4916.0,-6.0,3.3,0.0,1,0,2.150437e-04,...,0,0,0,0,1,0,0,0,0,0
4,5,107512,21.87,4916.0,-7.7,2.3,0.0,1,4,7.680134e-08,...,0,1,0,0,0,0,0,0,0,0


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29991 entries, 0 to 30590
Data columns (total 52 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   안전센터개수            29991 non-null  int64  
 1   인구수               29991 non-null  int64  
 2   면적                29991 non-null  float64
 3   인구밀도              29991 non-null  float64
 4   기온(°C)            29991 non-null  float64
 5   풍속(m/s)           29991 non-null  float64
 6   강수량(mm)           29991 non-null  float64
 7   month             29991 non-null  int64  
 8   hour              29991 non-null  int64  
 9   화재위험지수            29991 non-null  float64
 10  시군구_강남구           29991 non-null  uint8  
 11  시군구_강동구           29991 non-null  uint8  
 12  시군구_강북구           29991 non-null  uint8  
 13  시군구_강서구           29991 non-null  uint8  
 14  시군구_관악구           29991 non-null  uint8  
 15  시군구_광진구           29991 non-null  uint8  
 16  시군구_구로구           29991 non-null  uint8 

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [15]:
x =  data.drop(["화재위험지수"] , axis =1)
y = data["화재위험지수"]

피쳐간 정규화

In [16]:
scaler = MinMaxScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

In [19]:
X_train , X_test , Y_train ,y_test = train_test_split(x_scaled ,y, test_size = 0.1 , random_state = 5)
Y_train = Y_train.values.ravel()

In [20]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(y_test.shape)

(26991, 51)
(3000, 51)
(26991,)
(3000,)


In [21]:
model = RandomForestRegressor(n_estimators = 100 , max_depth = 10) #결정트리를 몇개 만들건지? #최대깊이는?

In [22]:
model.fit(X_train,Y_train)

RandomForestRegressor(max_depth=10)

In [23]:
model.predict(X_test)

array([2.65952976e-04, 4.39207533e-04, 2.49219200e-04, ...,
       1.39776710e-04, 5.70154587e-05, 1.50074516e-04])

In [24]:
model.score(X_test,y_test)

-0.09364392490323104

In [25]:
model.feature_importances_

array([2.54525680e-03, 4.64713884e-02, 2.03232766e-02, 5.15726781e-02,
       1.78932024e-01, 1.58002832e-01, 7.49916629e-03, 1.58067844e-01,
       2.60475253e-01, 5.23124719e-03, 1.71148839e-03, 3.56387667e-03,
       2.13254957e-03, 8.95875561e-04, 2.38214237e-06, 4.35642773e-04,
       4.87366105e-04, 1.16925253e-03, 1.91753623e-02, 6.89688000e-04,
       1.45212522e-03, 8.04634019e-04, 2.24530428e-03, 1.30072702e-03,
       9.39435840e-04, 3.43834105e-04, 1.00085784e-03, 3.46879817e-04,
       3.10341989e-03, 1.00455460e-03, 7.89966620e-04, 3.07411085e-03,
       7.28907073e-03, 6.70682441e-04, 6.41985860e-03, 2.86324235e-03,
       2.96682681e-07, 1.55206103e-03, 2.68424696e-03, 1.71450722e-03,
       3.61473743e-06, 3.10879380e-03, 1.42006663e-02, 2.44366654e-03,
       5.43530974e-03, 4.27374466e-03, 4.20158422e-03, 7.72044731e-11,
       7.33923395e-03, 0.00000000e+00, 9.11884915e-06])